In [1]:
# @title 1. Install & Fix Crash
!pip install -q gradio pandas scikit-learn google-generativeai joblib nest_asyncio

import nest_asyncio
import asyncio

# 🛠️ APPLY FIX: This prevents the "loop_factory" error in Colab
nest_asyncio.apply()

# Force asyncio to use a compatible runner
_original_run = asyncio.run
def _patched_run(main, *, loop_factory=None, **kwargs):
    return _original_run(main, **kwargs)
asyncio.run = _patched_run

print("✅ System Ready.")

✅ System Ready.


In [2]:
# @title 2. Train Models
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# 1. Load Data
df = pd.read_csv('crop_yield_dataset (1).csv')
df['Irrigation'] = df['Irrigation'].fillna('Unknown')
df['Previous_Crop'] = df['Previous_Crop'].fillna('Unknown')

# 2. Create Synthetic Price (Since it's missing)
# Logic: Lower Yield = Higher Price (Supply & Demand)
df['Price'] = (50000 / df['Yield_ton_per_ha']) + np.random.normal(0, 200, len(df))

# 3. Setup Training
X = df.drop(columns=['Yield_ton_per_ha', 'Price'])
y_yield = df['Yield_ton_per_ha']
y_price = df['Price']

# 4. Define Transformers (Handle Text & Numbers)
prep = ColumnTransformer([
    ('num', StandardScaler(), ['Soil_pH', 'Rainfall_mm', 'Temperature_C', 'Humidity_pct', 'Fertilizer_Used_kg', 'Pesticides_Used_kg', 'Planting_Density']),
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['Crop', 'Region', 'Soil_Type', 'Irrigation', 'Previous_Crop'])
])

# 5. Train!
yield_model = Pipeline([('p', prep), ('m', RandomForestRegressor(n_estimators=20))])
price_model = Pipeline([('p', prep), ('m', RandomForestRegressor(n_estimators=20))])

yield_model.fit(X, y_yield)
price_model.fit(X, y_price)

# 6. Save Options for Dropdowns
opts = {col: df[col].unique().tolist() for col in ['Crop', 'Region', 'Soil_Type', 'Irrigation', 'Previous_Crop']}

print("✅ Models Trained.")

✅ Models Trained.


In [3]:
# @title 3. Define Prediction Logic
def predict(crop, region, soil, irrig, prev, ph, rain, temp, humid, fert, pest, dens):
    # Create a single row of data
    row = pd.DataFrame([{
        'Crop': crop, 'Region': region, 'Soil_Type': soil,
        'Irrigation': irrig, 'Previous_Crop': prev,
        'Soil_pH': ph, 'Rainfall_mm': rain, 'Temperature_C': temp,
        'Humidity_pct': humid, 'Fertilizer_Used_kg': fert,
        'Pesticides_Used_kg': pest, 'Planting_Density': dens
    }])

    # Get answers
    y_pred = yield_model.predict(row)[0]
    p_pred = price_model.predict(row)[0]

    return f"{y_pred:.2f} tons/ha", f"₹ {p_pred:.2f} per ton"

In [4]:
# @title 4. Connect Groq AI Chatbot (Ultra Fast)
!pip install -q groq

from groq import Groq

# 👇 PASTE YOUR GROQ API KEY HERE 👇
GROQ_API_KEY = "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"

if GROQ_API_KEY and GROQ_API_KEY.startswith("gsk_"):
    try:
        client = Groq(api_key=GROQ_API_KEY)
        # Using Llama-3-70b or 8b for instant responses
        MODEL_NAME = "llama-3.3-70b-versatile"
        print("✅ Groq AI Connected! (Llama-3)")
    except Exception as e:
        print(f"❌ Connection Error: {e}")
        client = None
else:
    client = None
    print("⚠️ No valid Groq Key detected in Cell 4.")

def chat(msg, history):
    if not client:
        return "⚠️ Chatbot is offline. Please add your Groq API Key in Cell 4."

    # Using your actual dataset options to guide the AI
    system_prompt = f"You are an expert Agri-Advisor. Use this dataset context: {opts}."

    try:
        completion = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": msg}
            ],
            temperature=0.7,
            max_tokens=512
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"Groq Error: {e}")
        return "Groq is currently busy or the key is incorrect. Please try again."

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 3.2 MB/s eta 0:00:00
✅ Groq AI Connected! (Llama-3)


In [5]:
# @title 5. Launch UI
import gradio as gr

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🚜 SmartHarvest Engine")

    with gr.Tab("Predictor"):
        # Inputs
        with gr.Row():
            c1 = gr.Dropdown(opts['Crop'], label="Crop", value="Rice")
            c2 = gr.Dropdown(opts['Region'], label="Region")
            c3 = gr.Dropdown(opts['Soil_Type'], label="Soil")
            c4 = gr.Dropdown(opts['Irrigation'], label="Irrigation")
            c5 = gr.Dropdown(opts['Previous_Crop'], label="Prev Crop")

        with gr.Row():
            n1 = gr.Slider(4,9, label="pH"); n2 = gr.Slider(0,2000, label="Rain")
            n3 = gr.Slider(10,50, label="Temp"); n4 = gr.Slider(0,100, label="Humidity")
            n5 = gr.Number(100, label="Fertilizer"); n6 = gr.Number(20, label="Pest")
            n7 = gr.Number(5000, label="Density")

        btn = gr.Button("Predict Yield & Price", variant="primary")
        out = [gr.Textbox(label="Yield"), gr.Textbox(label="Price")]

        # Connect Button
        btn.click(predict, inputs=[c1,c2,c3,c4,c5,n1,n2,n3,n4,n5,n6,n7], outputs=out)

    with gr.Tab("Chat"):
        gr.ChatInterface(chat)

# Launch!
demo.launch(share=True, debug=False)

/tmp/ipython-input-2482919974.py:4: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7a1b894a14e965361e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
